_Data comes from **MIG/DualData**_

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import namedtuple
from argparse import ArgumentParser
from path import Path
from IPython.display import display

In [40]:
plt.style.use('ggplot')
plt.figure()
pd.set_option('display.max_columns', 50)

# File name / Environment / Crowd / AI
Simset = namedtuple("Simset", ["Name","Envs","Crowd","AI"])
boolmap = lambda x:bool(int(x))
Situs = [
    Simset("All", *map(boolmap,"111")),
    Simset("Environment", *map(boolmap,"100")),
    Simset("Crowd", *map(boolmap,"010")),
    Simset("AI", *map(boolmap,"001")),
]

current = Situs[0] 

In [43]:
def parseLogfile(filename):
    benchmark = pd.read_table(filename, sep=' ', skiprows=1)
    benchmark = benchmark[benchmark["frames"] < benchmark["frames"].max()]
    
    value_col_names = ["hashing", "time_avg", "len_avg", "ple_avg", "cls_avg"]
    col_names = [col_name for col_name in benchmark.columns if 
        (col_name.endswith("th_obstacle") and current.Envs) or
        (col_name.endswith("th_region") and current.Crowd) or
        (col_name.endswith("th_ai") and current.AI)]
    
    # Modifing & extracting datas
    benchmark.loc[:, "hashing"] = benchmark[col_names].apply(
        lambda x: hash(tuple(x)),axis=1)
    benchmark.loc[:, "time_avg"] = (
        benchmark["agent_time_enableds"].apply(
        lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
    benchmark.loc[:, "len_avg"] = (
        benchmark["agent_distance_traveleds"].apply(
        lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
    benchmark.loc[:, "ple_avg"] = (
        benchmark["agent_ple_energys"].apply(
        lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
    benchmark.loc[:, "cls_avg"] = (
        benchmark["collisionTimes"].apply(
        lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))

    sampleSet = benchmark[col_names + value_col_names]
    sample1, sample2 = sampleSet.iloc[::2], sampleSet.iloc[1::2]

    sample1.set_index('hashing', inplace=True)
    sample2.set_index('hashing', inplace=True)
    
    print("Dual trail")
    print( (sample1-sample2).dropna().apply(np.std, ddof=1)
      .loc[value_col_names[1:]].to_frame() )
    
    print("Varience test")
    print(sampleSet.apply(np.std, ddof=1).loc[value_col_names[1:]].to_frame())
    
    print("Mean Value test")
    print(sampleSet.apply(np.mean).loc[value_col_names[1:]].to_frame())

In [44]:
for file in Path("/home/kaidong/logfiles").glob("*Env*"):
    print(file) 
    print("-"*20)
    
    parseLogfile(file)
    print("\n"*2)

/home/kaidong/logfiles/MapA-SF-Env.log
--------------------
Dual trail
           0
time_avg NaN
len_avg  NaN
ple_avg  NaN
cls_avg  NaN
Varience test
                  0
time_avg   5.719305
len_avg    7.132128
ple_avg   24.146009
cls_avg    3.061273
Mean Value test
                   0
time_avg   84.217697
len_avg   102.188417
ple_avg   350.782949
cls_avg     4.929612



/home/kaidong/logfiles/MapB-ORCA-Env.log
--------------------
Dual trail
                 0
time_avg  1.387171
len_avg   1.287551
ple_avg   5.761329
cls_avg   1.479116
Varience test
                  0
time_avg   3.656902
len_avg    4.096479
ple_avg   14.608961
cls_avg    2.236466
Mean Value test
                   0
time_avg  107.380873
len_avg   136.671736
ple_avg   464.565396
cls_avg    14.206392



